In [ ]:
import sys
sys.path.append("D:/gnn-cloud-manufacturing/")

In [ ]:
import torch
from torch.optim import Adam
from tqdm.auto import trange
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import trange, tqdm
from cloudmanufacturing.data import read_fatahi_dataset
from cloudmanufacturing.mip_solver import mip_solve
from cloudmanufacturing.validation import objvalue, construct_delta
from cloudmanufacturing.graph import dglgraph_fixed, graph_gamma, os_type, ss_type, so_type
from cloudmanufacturing.graphconv import GNN
import dgl
from dgl.dataloading import GraphDataLoader
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import precision_recall_curve, auc
import pickle
import torch.nn.functional as F

## Загрузка данных и расчет оптимального маршрута

In [ ]:
dataset = read_fatahi_dataset('../data/fatahi.xlsx')

In [ ]:
DGList=[]
for problem in dataset[:2]:
    delta, train_gamma, status, value = mip_solve(problem)
    print(status, value)
    example_graph = dglgraph_fixed(problem, train_gamma)
    example_target = example_graph.edata['target'][os_type]
    example_graph.edata['feat'][os_type][:, 0] /= 10
    example_graph.edata['feat'][ss_type][:] /= 100
    DGList.append(example_graph)

In [11]:
# Запись в файл
with open('../data/fatahi_solve.xlsx', 'wb') as f:
    pickle.dump(DGList, f)

## Настройка обучения

In [ ]:
loader = GraphDataLoader(
    DGList,
    batch_size=2,
    drop_last=True,
    shuffle=True
)

In [ ]:
n_epochs = 200
num_batches = len(loader)
epoch_train_loss =0
batch_size=2
train_loss = []
train_objvalue = []
oper_max = 20
model = GNN(ins_dim=1, ino_dim=oper_max, out_dim=16, n_layers=1)
optim = Adam(model.parameters(), lr=0.01)

for i in range(n_epochs):
    for example_graph in tqdm(loader):
        logits = model(example_graph)
        example_target = example_graph.edata['target'][os_type]
        loss = F.binary_cross_entropy_with_logits(logits, example_target)
        batch_loss = loss.item()
        optim.zero_grad()
        batch_loss /= batch_size
        print(batch_loss)
        loss.backward()
        optim.step()
    
    train_loss.append(epoch_train_loss / num_batches)
    #train_objvalue.append(epoch_train_objvalue / num_batches)

In [ ]:
logits_np = logits.detach().numpy()
labels_np = example_target.detach().numpy()

precision, recall, thresholds = precision_recall_curve(labels_np, logits_np)
f1_scores = 2 * precision * recall / (precision + recall)

# Find the threshold that maximizes the F1-score
best_threshold = thresholds[np.argmax(f1_scores)]
print(f"Best threshold based on F1-score: {best_threshold:.2f}")

pr_auc = auc(recall, precision)
print(f"Area under the precision-recall curve: {pr_auc:.2f}")